In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import numpy as np
from graphdatascience import GraphDataScience
from py2neo import Graph
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import neo4jupyter
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops, to_dense_adj, dense_to_sparse
from torch_geometric.nn import VGAE,GAE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from torch_geometric.utils import to_dense_adj
#neo4jupyter.init_notebook_mode()
import tqdm as notebook_tqdm

In [4]:
import pickle
# Load the list of arrays from the .pkl file
with open('node_embeddings_no_cmr.pkl', 'rb') as pkl_file:
    extracted_data = pickle.load(pkl_file)

In [5]:
node_embeddings = extracted_data

In [ ]:
diseases=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Disease.csv')
gene=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Gene.csv')

In [ ]:
diseases_nodes=list(diseases.fname)
gene_nodes=list(gene.fname)

In [ ]:
diseases_embeddings = {key:node_embeddings[key] for key in diseases_nodes if key in node_embeddings}
gene_embeddings = {key:node_embeddings[key] for key in gene_nodes if key in node_embeddings}

### Gene-disease asscociation prediction MI

In [ ]:

All_Gene_nodes= pd.read_csv('C:\\Users\krjoob\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-afea5548-c3e4-4b02-b313-76660c29190a\\import\\Gene.csv')

In [11]:
list_all_gene = list(set(All_Gene_nodes.fname))

In [13]:
MI_gene_pos =['LDHA', 'MIR145', 'NR3C2', 'NPPA', 'TNNI3','AGT', 
              'PPARGC1A', 'EPO', 'MYH7', 'BRAP', 'PDE3A', 'SH2B3',
              'S100B', 'BAK1', 'SOD2', 'KLF13', 'BECN1', 'ADORA1', 
              'F13A1', 'DGKZ', 'MIR125B2', 'BCL2','APOE', 'TGFB1', 
              'NOS3', 'P2RY12', 'ATM', 'MFF', 'GGT1', 'LRP8', 'MMP2', 
              'TM6SF2','PHACTR1', 'F2', 'GSK3B', 'WDR12', 'NOS2', 'ACTA2', 
              'BCL2L1', 'ICAM1', 'PSMA6', 'MMP9','ADORA3', 'DDIT3','REN', 
              'SOD1', 'PLAT', 'LGALS3', 'IL1RN', 'IL1B', 'ITGB3', 'HSPA5', 
             'PLAU', 'SELL', 'CASP3', 'F5', 'PAPPA','NRIP1', 'ESR1', 'THBD',
              'LTA', 'CKM', 'HP','F7', 'ATP1A1', 'CKB', 'CAT', 'MIA3', 
              'PRKCE', 'GATA4', 'GCLM','KLK1','HMGB1', 'KIF6',  'HSD11B2', 
               'MIR423', 'TNF', 'IL10', 'OLR1', 'LGALS2', 'MIR761', 'IFNA2',
              'BAX', 'ACE', 'MIAT', 'GCLC', 'CREB1', 'CREM','PLN', 'TNFSF4',
              'TNFRSF12A', 'IL6','TIMP2', 'VDAC1', 'DAB2IP',  
]

In [90]:
MI_gene_neg = ['SPATA13', 'PRDM12', 'VWA8', 'PAGR1', 'S1PR2', 'CHAMP1', 'WFDC9', 'EP400P1', 'FAM131A', 'KLHL3',
         'AHSA1', 'KCNMB2', 'SLC25A31', 'SIKE1', 'DUOXA1', 'CHTF8', 'GSTT2', 'OSBPL7', 'PCF11', 'TLR7',
         'AAGAB', 'MGST1', 'MAMDC2', 'DNAL4', 'VAX2', 'IGKV4-1', 'CLP1', 'UNC79', 'CYP46A1', 'CRYGS',
         'CRACR2B', 'ZNF347', 'NR2F1', 'ADAMTS4', 'ISCU', 'FAM72A', 'AGPAT5', 'KIF3B', 'CABCOCO1', 'ARMC2',
         'SMIM2', 'CALCOCO1', 'ETHE1', 'HRC', 'PEX13', 'SH3BGR', 'NFASC', 'CYP27C1', 'MED12L', 'NBAS',
         'SOWAHC', 'ALDOB', 'ABTB1', 'TMIGD3', 'FREM1', 'BBIP1', 'PATE1', 'GMEB1', 'OPA3', 'TAS2R13', 'ABR',
         'C5orf34', 'ARMC12', 'RFLNA', 'TCN1', 'PDZD11', 'SWAP70', 'TEP1', 'COX6A2', 'HINT1', 'SCARNA27',
         'HINT3', 'TTC8', 'PSG2', 'DHRS9', 'ITFG2', 'HABP2', 'USB1', 'NSMCE1', 'TAS2R7', 'SLITRK4', 'CNPY1',
         'C9orf153', 'IGHV4-31', 'POC1B-GALNT4', 'METTL17', 'TUBD1', 'ESCO2', 'TMEM54', 'ANKRD53']

In [67]:
list_of_pos_neg = MI_gene_pos+MI_gene_neg

In [68]:
list_uknown_genes = [item for item in list_all_gene if item not in list_of_pos_neg]

In [71]:
gene_embeddigns_pos = [np.concatenate((gene_embeddings[key],diseases_embeddings.get('myocardial infarction'))) for key in MI_gene_pos if key in gene_embeddings]
gene_embeddigns_neg = [np.concatenate((gene_embeddings[key],diseases_embeddings.get('myocardial infarction'))) for key in MI_gene_neg if key in gene_embeddings]
uknown_gene_embeddigns = [np.concatenate((gene_embeddings[key],diseases_embeddings.get('myocardial infarction'))) for key in list_uknown_genes if key in gene_embeddings]
MI_embeddings = diseases_embeddings.get('myocardial infarction')

In [74]:
gene_disease_pos = pd.DataFrame(np.array(gene_embeddigns_pos))
gene_disease_pos['Label'] = 1
gene_disease_neg = pd.DataFrame(np.array(gene_embeddigns_neg))
gene_disease_neg['Label'] = 0

In [75]:
gene_disease_pos_neg = pd.concat([gene_disease_pos,gene_disease_neg])

In [76]:
uknown_genes_names= [key for key in list_uknown_genes if key in gene_embeddings]
uknown_gene_embeddigns_df = pd.DataFrame(np.array(uknown_gene_embeddigns))

In [77]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
import random 

random.seed(123456)
np.random.seed(123456)
X = gene_disease_pos_neg.iloc[:,0:-1]
y = gene_disease_pos_neg.iloc[:,-1]
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=20)


In [79]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
# Define the parameter grid
param_grid = {
    'C': [0.1, 1,5, 10, 50, 100, 200, 300, 500],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Initialize the classifier
random.seed(123)
svm = SVC(probability=True, random_state=42)

# Initialize GridSearchCV
random.seed(123)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print("Best parameters found: ", best_params)

# Evaluate the best estimator
y_pred_svm = best_svm.predict(X_test)
y_pred_proba_svm = best_svm.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_svm))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
Accuracy: 0.8108108108108109
ROC AUC: 0.8173076923076925


In [80]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

random.seed(123)
# Define the tuned parameters (example parameters, replace with your actual tuned values)
tuned_params = {
    'C': 100,        # Regularization parameter
    'gamma': 1, # Kernel coefficient
    'kernel': 'rbf' # Kernel type
}

# Initialize the classifier with tuned parameters
svm = SVC(probability=True, random_state=42, **tuned_params)

# Train the classifier
svm.fit(X_train, y_train)

# Predict the test set labels
y_pred_svm = svm.predict(X_test)
y_pred_proba_svm = svm.predict_proba(X_test)[:, 1]

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_svm))


Accuracy: 0.8108108108108109
ROC AUC: 0.8173076923076925


In [84]:
svm_my_dict = {}
for i in range(0,uknown_gene_embeddigns_df.shape[0]):
    y_pred_svm_p = svm.predict(uknown_gene_embeddigns_df.iloc[i:i+1,:])
    y_pred_proba_svm_p = svm.predict_proba(uknown_gene_embeddigns_df.iloc[i:i+1,:])[:, 1]
    print(uknown_genes_names[i],y_pred_svm_p[0],y_pred_proba_svm_p[0])
    if y_pred_svm_p[0]>0:
        svm_my_dict[uknown_genes_names[i]] =y_pred_proba_svm_p[0]

pos_prd_prob_svm = dict(sorted(svm_my_dict.items(), key=lambda item: item[1],reverse=True))

GPR35 1 0.9999954916823102
MRPL27 0 0.44633602809374406
LAMTOR1 1 0.7812830449439464
WDR4 0 0.48393779354437394
METRNL 0 0.2618013578451438
FAM149B1 0 0.24135862313348835
NEFH 0 0.26004273181247434
ARID3A 1 0.8629989390449482
PSENEN 0 0.30313024139366
TPI1P2 0 0.2560618812142737
ADAMDEC1 0 0.5126399882307053
OR4D6 0 0.26022658525668535
AURKAIP1 1 0.8425661906075879
TMSB4XP4 0 0.2545284092150466
CXCL10 1 0.6491155816774806
PHACTR4 0 0.36392111641944336
WDR7 0 0.33279677152298937
NUP205 1 0.574306957653509
BET1 1 0.9942053317485521
FTSJ3 1 0.9670902041320616
ST8SIA1 0 0.23743756004586988
MRPS24 0 0.45976622999809375
WDR24 1 0.6889844677013317
RRP1 1 0.6544147121789691
DUSP26 0 0.26278658560059726
PAFAH2 0 0.29716978546132417
METTL7A 0 0.24745006671818945
IRF2BP2 1 0.6479613619457001
ATG101 0 0.35678617188064676
PGAP2 1 0.8466743118669249
DAD1 0 0.514499386747878
CYP1A2 0 0.394606091395363
CARTPT 0 0.24064163078635917
TLE1 1 0.9999999382853666
MSGN1 0 0.5325832351784732
NAP1L4 0 0.5434526